In [67]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt


The code from google.colab import drive imports the drive module from Google Colab, allowing you to interact with your Google Drive files.

The drive.mount('/content/drive') command mounts your Google Drive to the Colab environment, providing access to your files stored in Google Drive. Once mounted, you can read, write, and manipulate files in Google Drive directly from your Colab notebook. The mounted drive is accessible at the path /content/drive.

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The variable file_path stores the location of the file D.csv within your Google Drive. The path "/content/drive/MyDrive/Reduce vehicle count and grouping societies/D.csv" points to the file in the mounted Google Drive directory, allowing you to access it from your Colab environment.

In [69]:
file_path = "/content/drive/MyDrive/Reduce vehicle  count and grouping societies/D.csv"  # Replace with your file path

data = pd.read_csv(file_path, sep=",") loads the CSV file from file_path into a Pandas DataFrame, using commas as the separator.Print the data.



In [70]:
data = pd.read_csv(file_path,sep=",")

In [102]:
print(data.head(5))

           Society Name   Latitude  Longitude  Cluster
0    Greenfield Heights  28.704131  77.053478        0
1           Emerald Bay  28.738793  77.071840        0
2   Pinewood Residences  28.703083  77.108263        0
3  Moonlight Apartments  28.711415  77.078376        0
4     Sunny Side Villas  28.703065  77.099209        0


coords = data[['Latitude', 'Longitude']].to_numpy() extracts the 'Latitude' and 'Longitude' columns from the DataFrame data and converts them into a NumPy array.

In [72]:
coords = data[['Latitude', 'Longitude']].to_numpy()

from geopy.distance import great_circle imports the great_circle function from the geopy library, which calculates the shortest distance between two points on the Earth's surface, using their latitude and longitude coordinates.

In [73]:
from geopy.distance import great_circle

The code performs the following actions:

Set clustering radius: epsilon = 2 / kms_per_radian sets the radius for clustering to 2 km, converting it to radians. The variable kms_per_radian likely holds the number of kilometers per radian of latitude.

DBSCAN Clustering: The DBSCAN algorithm is applied to the coordinates (coords) to identify clusters:

eps=epsilon sets the radius for grouping points.
min_samples=2 specifies the minimum number of points needed to form a cluster.
algorithm='ball_tree' selects the algorithm used for the distance computation.
metric='haversine' uses the Haversine formula to calculate distances between points on the Earth's surface, considering the curvature of the Earth.
Fit model: .fit(np.radians(coords)) applies DBSCAN to the coordinates, first converting them from degrees to radians since the Haversine distance metric requires radians.

In [74]:
kms_per_radian = 6371.0088

# Set clustering radius to 3 km (convert to radians)
epsilon = 2 / kms_per_radian
# Assuming coords is a numpy array with latitude and longitude
db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))

data['Cluster'] = db.labels_ assigns the cluster labels obtained from the DBSCAN algorithm (db.labels_) to a new column called 'Cluster' in the data DataFrame. Each point in the DataFrame will be labeled with its corresponding cluster, where:

A label of -1 indicates noise (points that don't belong to any cluster).
Other values represent the cluster number assigned to each point.

In [75]:
data['Cluster'] = db.labels_

In [103]:
print(data.head(5))

           Society Name   Latitude  Longitude  Cluster
0    Greenfield Heights  28.704131  77.053478        0
1           Emerald Bay  28.738793  77.071840        0
2   Pinewood Residences  28.703083  77.108263        0
3  Moonlight Apartments  28.711415  77.078376        0
4     Sunny Side Villas  28.703065  77.099209        0


Define output path: output_path = "clustered_data.xlsx" specifies the file path where the clustered data will be saved, in this case, as an Excel file.

Save DataFrame to Excel: data.to_excel(output_path, index=False) saves the data DataFrame (with the added 'Cluster' column) to an Excel file at the specified output_path. The index=False argument prevents the DataFrame's index from being written to the Excel file.

Print confirmation: print(f"Clustered data saved to {output_path}") outputs a message confirming the file has been saved, with the specified path.

In [77]:
output_path = "clustered_data.xlsx"
data.to_excel(output_path, index=False)
print(f"Clustered data saved to {output_path}")

Clustered data saved to clustered_data.xlsx


The code creates an interactive scatter plot using Plotly Express. It:

Plots Longitude on the x-axis and Latitude on the y-axis.
Colors the points based on the Cluster column.
Displays the Society Name and Cluster on hover.
Adds a title "Spread of Societies" and labels for the axes.
Finally, fig.show() renders the plot.

In [78]:
import plotly.express as px
import pandas as pd

# Create an interactive scatter plot
fig = px.scatter(
    data,
    x='Longitude',
    y='Latitude',
    color='Cluster',
    hover_data=['Society Name', 'Cluster'],
    title='Spread of Societies',
    labels={'Longitude': 'Longitude', 'Latitude': 'Latitude'}
)

# Show plot
fig.show()

In [79]:
df=data

Filter out noise points: df = df[df["Cluster"] >= 0] filters the DataFrame to include only rows where the 'Cluster' column has values greater than or equal to 0, effectively removing noise points (labeled as -1 by DBSCAN).

Rename column: df = df.rename(columns={"Society Name": "Society_Name"}) renames the column "Society Name" to "Society_Name" for consistency or style.

Display first 5 rows: print(df.head(5)) prints the first 5 rows of the modified DataFrame.

This results in a cleaned and renamed DataFrame with no noise points, ready for further analysis or visualization.

In [80]:
#Let's filter this data and remove the societies that are outliers.
df=df[df["Cluster"]>=0]
df=df.rename(columns={"Society Name":"Society_Name"})
print(df.head(5))

           Society_Name   Latitude  Longitude  Cluster
0    Greenfield Heights  28.704131  77.053478        0
1           Emerald Bay  28.738793  77.071840        0
2   Pinewood Residences  28.703083  77.108263        0
3  Moonlight Apartments  28.711415  77.078376        0
4     Sunny Side Villas  28.703065  77.099209        0


The code performs the following actions:

Concatenate Society_Name and Latitude: df.loc[:, 'cc'] = df["Latitude"].astype(str) + df["Society_Name"] creates a new column 'cc' by concatenating the Latitude (converted to a string) and Society_Name. This combines both values into a single string for each row.

Display first 5 rows: print(df.head(5)) prints the first 5 rows of the DataFrame, now with the additional 'cc' column.

This results in a new column 'cc', where each entry contains the Latitude followed by the corresponding Society_Name.

In [81]:
#lets concatenate society  name and latitude
df.loc[:, 'cc']  =df["Latitude"].astype(str) +df["Society_Name"]
print(df.head(5))

           Society_Name   Latitude  Longitude  Cluster  \
0    Greenfield Heights  28.704131  77.053478        0   
1           Emerald Bay  28.738793  77.071840        0   
2   Pinewood Residences  28.703083  77.108263        0   
3  Moonlight Apartments  28.711415  77.078376        0   
4     Sunny Side Villas  28.703065  77.099209        0   

                                cc  
0    28.70413068Greenfield Heights  
1           28.73879327Emerald Bay  
2   28.70308302Pinewood Residences  
3  28.71141539Moonlight Apartments  
4      28.7030651Sunny Side Villas  


In [82]:
df1 = pd.read_csv("/content/drive/MyDrive/Reduce vehicle  count and grouping societies/societies_orders.csv",sep=",")
df1=df1.rename(columns={"Society Name":"Society_Name"})
print(df1.head(10))

           Society_Name                               cc   Latitude  \
0    Greenfield Heights    28.70413068Greenfield Heights  28.704131   
1           Emerald Bay           28.73879327Emerald Bay  28.738793   
2   Pinewood Residences   28.70308302Pinewood Residences  28.703083   
3  Moonlight Apartments  28.71141539Moonlight Apartments  28.711415   
4     Sunny Side Villas      28.7030651Sunny Side Villas  28.703065   
5    Greenfield Heights    28.70267864Greenfield Heights  28.702679   
6     Sunny Side Villas     28.68872911Sunny Side Villas  28.688729   
7           Emerald Bay           28.69450457Emerald Bay  28.694505   
8        Evergreen Park        28.69165389Evergreen Park  28.691654   
9    Blue Ridge Society    28.68737727Blue Ridge Society  28.687377   

   Longitude  Orders  
0  77.053478      16  
1  77.071840      15  
2  77.108263      15  
3  77.078376      25  
4  77.099209       9  
5  77.063366       1  
6  77.102718       2  
7  77.126680      21  
8  77.07940

The code performs the following actions:

Create an in-memory SQLite database:
conn = sqlite3.connect(':memory:') creates an in-memory SQLite database, meaning the database exists only during the runtime of the program.

Create a cursor:
cursor = conn.cursor() initializes a cursor to interact with the SQLite database.

Load DataFrames into SQLite:

df.to_sql('df', conn, index=False, if_exists='replace') stores the DataFrame df in the in-memory SQLite database as a table named df. If the table already exists, it will be replaced.
df1.to_sql('df1', conn, index=False, if_exists='replace') does the same for the DataFrame df1, creating a table named df1.
Execute SQL Query:
query = "SELECT df.Society_Name, df.Latitude, df.Longitude, df.Cluster, Orders FROM df JOIN df1 ON df.cc = df1.cc" is an SQL query that performs an inner join between the two tables (df and df1) on the 'cc' column, selecting specific columns from both tables.

Fetch and display results:
result = pd.read_sql(query, conn) runs the query and loads the results into a Pandas DataFrame result.
print(result.head(55)) prints the first 55 rows of the query result.

This process simulates querying a database using SQLite with two DataFrames (df and df1) stored in an in-memory database, where data from both tables is joined based on the 'cc' column.

In [97]:
#An Excel file was used as a prototype instead of the actual database to address privacy concerns."
import sqlite3

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Load DataFrame into SQLite
df.to_sql('df', conn, index=False, if_exists='replace')

df1.to_sql('df1', conn, index=False, if_exists='replace')
# Example Query to Fetch All Columns

query = "SELECT df.Society_Name,df.Latitude,df.Longitude,df.Cluster,Orders FROM df join df1 on df.cc=df1.cc"

result = pd.read_sql(query, conn)

# Display the query result
print(result.head(55))

            Society_Name   Latitude  Longitude  Cluster  Orders
0     Greenfield Heights  28.704131  77.053478        0      16
1            Emerald Bay  28.738793  77.071840        0      15
2    Pinewood Residences  28.703083  77.108263        0      15
3   Moonlight Apartments  28.711415  77.078376        0      25
4      Sunny Side Villas  28.703065  77.099209        0       9
5     Greenfield Heights  28.702679  77.063366        0       1
6      Sunny Side Villas  28.688729  77.102718        0       2
7            Emerald Bay  28.694505  77.126680        0      21
8         Evergreen Park  28.691654  77.079409        0      16
9     Blue Ridge Society  28.687377  77.103339        0      24
10      Rosewood Society  28.698388  77.064722        0       2
11    Palm Grove Estates  28.666351  77.109679        0       3
12        Evergreen Park  28.683959  77.125922        0       3
13       Maple Residency  28.751879  77.086308        0       4
14        Golden Springs  28.737199  77.

In [100]:
result
print(type(result))

<class 'pandas.core.frame.DataFrame'>


The code result1 = result["Orders"].sum() calculates the sum of the values in the "Orders" column of the result DataFrame.

The variable result1 will hold the total sum of all the orders in that column.

In [101]:
result1=result["Orders"].sum()
result1

532

while Total orders are 532.

A Bolero can carry up to 300 orders, while a Tata Ace can handle 250 orders. Since the total number of orders exceeds 300, two Boleros can be deployed to deliver the orders.
